## Train BERT model

### Load the dataset

In [1]:
import json

train_set = json.load(open("cord19/train_set.json", "r"))

In [2]:
train_set[0]

{'query_id': 1,
 'query': 'coronavirus origin',
 'relevant_docs': [{'id': '00fmeepz', 'score': 1},
  {'id': '021q9884', 'score': 1},
  {'id': '02f0opkr', 'score': 1},
  {'id': '08ds967z', 'score': 1},
  {'id': '0be4wta5', 'score': 2},
  {'id': '0chuwvg6', 'score': 2},
  {'id': '0e1w86tg', 'score': 1},
  {'id': '0hnh4n9e', 'score': 1},
  {'id': '0iq9s94n', 'score': 1},
  {'id': '0ne21in2', 'score': 1},
  {'id': '0nh58odf', 'score': 2},
  {'id': '0v5wo0ty', 'score': 1},
  {'id': '0xhho1sh', 'score': 2},
  {'id': '0xruezf2', 'score': 1},
  {'id': '0y34yxlb', 'score': 2},
  {'id': '105q161g', 'score': 2},
  {'id': '10ecm4wi', 'score': 1},
  {'id': '11edrkav', 'score': 2},
  {'id': '12dcftwt', 'score': 2},
  {'id': '13ir7swr', 'score': 2},
  {'id': '1abp6oom', 'score': 2},
  {'id': '1bvsn9e8', 'score': 2},
  {'id': '1c47w4q5', 'score': 2},
  {'id': '1eytelxn', 'score': 1},
  {'id': '1ihz8z6q', 'score': 1},
  {'id': '1p7fz9zr', 'score': 1},
  {'id': '1qkwsh6a', 'score': 2},
  {'id': '1s2zsqo

In [3]:
from pandas import read_csv

training_data_batch = read_csv("cord19/training_data_bert_or_bm25_10_total.csv")

In [4]:
training_data_batch.head()

,document_id,query_id,label,title-full
0,00fmeepz,1,1,Comprehensive overview of COVID-19 based on cu...
1,f047dvb4,1,0,Retrouver les origines du SARS-CoV-2 dans les ...
2,bc8taaw9,1,0,[Tracing the origins of SARS-COV-2 in coronavi...
3,12dcftwt,1,0,Possible Bat Origin of Severe Acute Respirator...
4,kqqantwg,1,0,Possible Bat Origin of Severe Acute Respirator...


### Create Datasets

Create a list for queries, docs and labels.

In [ ]:
train_queries = []
train_docs = []
train_labels = []
for data_point in train_set:
    query_id = data_point["query_id"]
    titles = training_data_batch[training_data_batch["query_id"] == query_id]["title-full"].tolist()
    train_docs.extend(titles)
    train_labels.extend([1 if x > 0 else 0 for x in training_data_batch[training_data_batch["query_id"] == query_id]["label"].tolist()])
    query = data_point["query"]
    train_queries.extend([query] * len(titles))

Split training data into train and validation sets.

In [ ]:
from sklearn.model_selection import train_test_split
train_queries, val_queries, train_docs, val_docs, train_labels, val_labels = train_test_split(
    train_queries, train_docs, train_labels, test_size=.2
)

Ensure training encodings are compatible with encodings used at serving time.

In [ ]:
def create_bert_encodings(queries, docs, tokenizer, query_input_size, doc_input_size):
    queries_encodings = tokenizer(
        queries, truncation=True, max_length=query_input_size-2, add_special_tokens=False
    )
    docs_encodings = tokenizer(
        docs, truncation=True, max_length=doc_input_size-1, add_special_tokens=False
    )
    
    TOKEN_NONE=0
    TOKEN_CLS=101
    TOKEN_SEP=102

    input_ids = []
    token_type_ids = []
    attention_mask = []
    for query_input_ids, doc_input_ids in zip(queries_encodings["input_ids"], docs_encodings["input_ids"]):
        # create input id
        input_id = [TOKEN_CLS] + query_input_ids + [TOKEN_SEP] + doc_input_ids + [TOKEN_SEP]
        number_tokens = len(input_id)
        padding_length = max(128 - number_tokens, 0)
        input_id = input_id + [TOKEN_NONE] * padding_length
        input_ids.append(input_id)
        # create token id
        token_type_id = [0] * len([TOKEN_CLS] + query_input_ids + [TOKEN_SEP]) + [1] * len(doc_input_ids + [TOKEN_SEP]) + [TOKEN_NONE] * padding_length
        token_type_ids.append(token_type_id)
        # create attention_mask
        attention_mask.append([1] * number_tokens + [TOKEN_NONE] * padding_length)

    encodings = {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask
    }
    return encodings

Create train and validation encodings.

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = create_bert_encodings(
    queries=train_queries, 
    docs=train_docs, 
    tokenizer=tokenizer, 
    query_input_size=64, 
    doc_input_size=64
)

val_encodings = create_bert_encodings(
    queries=val_queries, 
    docs=val_docs, 
    tokenizer=tokenizer, 
    query_input_size=64, 
    doc_input_size=64
)

Create a torch dataset

In [ ]:
import torch

class Cord19Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Cord19Dataset(train_encodings, train_labels)
val_dataset = Cord19Dataset(val_encodings, val_labels)

Fine-tune the model (only task specific weights.)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
for param in model.base_model.parameters():
    param.requires_grad = False

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

### Export the model to onnx

In [12]:
from torch.onnx import export
from pathlib import Path 

model_onnx_path = Path("bert.onnx")
dummy_input = (
    train_dataset[0]["input_ids"].unsqueeze(0), 
    train_dataset[0]["token_type_ids"].unsqueeze(0), 
    train_dataset[0]["attention_mask"].unsqueeze(0)
)
input_names = ["input_ids", "token_type_ids", "attention_mask"]
output_names = ["logits"]
export(
    model, dummy_input, model_onnx_path, input_names = input_names, 
    output_names = output_names, verbose=False, opset_version=11
)

Check output type.

In [1]:
import onnxruntime as ort
m = ort.InferenceSession("bert.onnx") 
print(m.get_outputs()[0].name)
print(m.get_outputs()[0].type)
print(m.get_outputs()[0].shape)

logits
tensor(float)
[1, 2]


Another form to check output type.

In [2]:
import onnx                                                                                                                                                                          
m = onnx.load("bert.onnx")                                                                                                                                                         
m.graph.output

[name: "logits"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 2
      }
    }
  }
}
]